## 영화 인물 관련 정보(감독, 등장인물 등) 기반 추천 모델 구현

### 0) 데이터셋 설명
#### tmdb_movies
- id : 각 영화에 대한 고유 ID
- title : 영화 제목
- runtime : 상영 시간
- genres : 영화 장르
- overview : 영화에 대한 간략한 설명
- popularity : TMDB에서 제공하는 인기도
- vote_avearage : TMDB에서 받은 평점 평균
- vote_count : TMDB에서 받은 투표수

#### tmdb_credits
- movie_id : 각 영화에 대한 고유 ID
- cast : 모든 출연진
- crew : 모든 제작진

In [ ]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies = pd.read_parquet('dataset/tmdb_movies.parquet')
credits = pd.read_parquet('dataset/tmdb_credits.parquet')

print(f"movies: {movies.shape}")
display(movies.head())

print(f"credits: {credits.shape}")
display(credits.head())

### 1) 데이터 전처리

In [ ]:
df = movies.merge(credits, on='id')

In [ ]:
# 필요 컬럼 추출

info_col = ['genres', 'cast', 'crew']

df = df[info_col]

df.head()

In [ ]:
# 각 컬럼의 문자열을 list 형식으로 변환

for col in info_col:
    df[col] = df[col].apply(eval)
    
df.dtypes

### 2) genre 컬럼 전처리

In [ ]:
# 장르 추출하여 'genre_names' 컬럼에 저장

def get_genre_names(val):
        return ' '.join([i.get('name','').lower() for i in val])

df['genres'] = df['genres'].apply(get_genre_names)

In [ ]:
df.head()

### 3) cast 컬럼 전처리

In [ ]:
import re

# cast 컬럼의 값에서 name 키만 추출하여 빈칸 기준으로 join하는 함수
def extract_cast(val):
    # val이 리스트가 아닐 경우(결측치 등) 예외 처리
    if not isinstance(val, list):
        return_val =  ""
    
    # name 키의 값을 추출하여 빈킨 기준으로 join
    names = [i.get('name', '').lower().strip().replace(" ", "") for i in val]
    
    # 알파벳과 숫자를 제외한 모든 문자를 제거하는 정규표현식 적용(특수문자, 공백 등을 모두 제거)
    names = [re.sub(r'[^a-zA-Z0-9]', '',i) for i in names]
    
    # 등장 인물 6명까지만 포함
    if len(names) > 6:
        names = names[:6]
    
    return_val = ' '.join(names)
    
    return return_val
    
df['cast'] = df['cast'].apply(extract_cast)

In [ ]:
df.head(5)

### 4) crew 컬럼 전처리

In [ ]:
print(df['crew'].iloc[10])

In [ ]:
# crew 컬럼에서 director 키의 값만 추출
def extract_crew(val):
    # val이 리스트가 아닐 경우(결측치 등) 예외 처리
    if not isinstance(val, list):
        return ""
    for i in val:
        if i['job'] == 'Director':
            return i['name'].lower().strip().replace(" ", "")

df['crew'] = df['crew'].apply(extract_crew)

In [ ]:
df.head()

### 5) feature 컬럼 생성 (genre + cast + crew)

In [ ]:
# genres, cast, crew 컬럼을 빈칸 기준으로 합쳐서 feature 컬럼 생성
df['feature'] = df['genres'].astype(str) + ' ' + df['cast'].astype(str) + ' ' + df['crew'].astype(str)

df.head()

### 6) 데이터 벡터화

In [ ]:
## TF-IDF 벡터화

tfidf = TfidfVectorizer(
    lowercase=True,
)
tfidf_matrix = tfidf.fit_transform(df['feature'])

tfidf_matrix.shape

### 7) 유사도 계산

In [ ]:
info_cos_sim = cosine_similarity(tfidf_matrix)

info_cos_sim.shape

### 8) 추천 함수 생성

In [ ]:
def info_recommendation(
    dataframe: pd.DataFrame, 
    movie_name: str,
    sim_matrix,
    top_n: int,
    ) -> pd.Series :

    # movie_name이 대소문자 구분 없이 필터링되도록 처리
    idx = dataframe[dataframe['title'].str.lower() == movie_name.lower()].index[0]

    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

    movie_idx = [i[0] for i in sim_scores if i[0] != idx][:top_n]

    return dataframe['title'].iloc[movie_idx]

movie_name = 'titanic'

info_recommendation(
    dataframe=movies,
    movie_name=movie_name,
    sim_matrix=info_cos_sim,
    top_n=10
)

### 9) 유사도 행렬 저장

In [ ]:
joblib.dump(info_cos_sim, "models/info_cos_sim.pkl")